In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from plot_utils import plot_3d_scatter
import QAOA_paper_proxy

%matplotlib widget

def test_plot_number_with_cost(num_constraints: int, num_qubits: int) -> None:
    costs = np.linspace(0, num_constraints, num_constraints+1)
    num_with_costs = np.zeros(len(costs))
    for i in range (len(costs)):
        num_with_costs[i] = QAOA_paper_proxy.number_with_cost_paper_proxy(costs[i], num_constraints, num_qubits)
    
    fig = plt.figure()
    ax = fig.add_subplot()
    ax.scatter(costs, num_with_costs)
    
def test_plot_by_cost_2(cost: int, num_constraints: int, num_qubits: int) -> None:
    cost_2s = np.linspace(0, num_constraints, num_constraints+1, dtype=int)
    distances = np.linspace(0, num_qubits, num_qubits+1, dtype=int)
    num_of_costs_at_distances = np.zeros([len(cost_2s), len(distances)])
    for i in range(len(cost_2s)):
        for j in range(len(distances)):
            num_of_costs_at_distances[i][j] = QAOA_paper_proxy.number_of_costs_at_distance_paper_proxy(cost, cost_2s[i], distances[j], num_constraints, num_qubits)
    
    plot_3d_scatter(cost_2s, distances, num_of_costs_at_distances, f"\"Approximate\" N({cost};d,c) from paper", "cost_2", "distance", "proxy")

test_plot_number_with_cost(20, 10)
test_plot_by_cost_2(4, 12, 6)
plt.show()

In [ ]:
from QAOA_paper_proxy import QAOA_paper_proxy
from plot_utils import plot_heat_map
import matplotlib.pyplot as plt
import numpy as np


def collect_parameter_data(num_constraints: int, num_qubits: int, gammas: np.ndarray, betas: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    expectations = np.zeros([len(gammas), len(betas)])

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            _, expectations[i][j] = QAOA_paper_proxy(1, gamma, beta, num_constraints, num_qubits)

    return expectations


gammas = np.linspace(0, np.pi/2, 50)
betas = np.linspace(0, np.pi, 50)
overlaps = collect_parameter_data(10, 6, gammas, betas)
plot_heat_map(gammas, betas, overlaps, "Expectation Proxy from paper", "Gamma", "Beta")

plt.show()

In [ ]:
from QAOA_proxy import QAOA_proxy
from plot_utils import plot_heat_map
import matplotlib.pyplot as plt
import numpy as np


def collect_parameter_data(num_constraints: int, num_qubits: int, gammas: np.ndarray, betas: np.ndarray):
    expectations = np.zeros([len(gammas), len(betas)])
    amplitudes = np.zeros([num_constraints + 1, len(gammas), len(betas)])

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            amps, expectations[i][j] = QAOA_proxy(1, gamma, beta, num_constraints, num_qubits)
            for c in range(num_constraints+1):
                amplitudes[c][i][j] = abs(amps[1][c]) ** 2

    return amplitudes, expectations


gammas = np.linspace(0, np.pi/2, 50)
betas = np.linspace(0, np.pi, 50)
amplitudes, overlaps = collect_parameter_data(21, 10, gammas, betas)

for c in range(0, 21):
    plot_heat_map(gammas, betas, amplitudes[c], f"Amplitude for c={c}", "Gamma", "Beta")

plot_heat_map(gammas, betas, overlaps, f"Expectation Proxy", "Gamma", "Beta")

plt.show()

In [ ]:
from QAOA_paper_proxy import QAOA_paper_proxy
from plot_utils import plot_heat_map
import matplotlib.pyplot as plt
import numpy as np


def collect_parameter_data(num_constraints: int, num_qubits: int, gammas: np.ndarray, betas: np.ndarray):
    expectations = np.zeros([len(gammas), len(betas)])
    amplitudes = np.zeros([num_constraints + 1, len(gammas), len(betas)])

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            amps, expectations[i][j] = QAOA_paper_proxy(1, gamma, beta, num_constraints, num_qubits)
            for c in range(num_constraints+1):
                amplitudes[c][i][j] = abs(amps[1][c]) ** 2

    return amplitudes, expectations


gammas = np.linspace(0, np.pi/2, 25)
betas = np.linspace(0, np.pi, 25)
amplitudes, overlaps = collect_parameter_data(21, 10, gammas, betas)

for c in range(0, 21):
    plot_heat_map(gammas, betas, amplitudes[c], f"Amplitude for c={c}", "Gamma", "Beta")

plot_heat_map(gammas, betas, overlaps, f"Expectation Proxy", "Gamma", "Beta")

plt.show()

In [ ]:
from QAOA_simulator import get_simulator, get_overlap, get_result
from plot_utils import plot_heat_map
import networkx as nx
import qokit.maxcut as mc
import numpy as np

def collect_parameter_data(G: nx.Graph, gammas: np.ndarray, betas: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    overlaps = np.zeros([len(gammas), len(betas)])
    N = G.number_of_nodes()
    ising_model = mc.get_maxcut_terms(G)
    sim = get_simulator(N, ising_model)

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            result = get_result(N, ising_model, gamma, beta, sim)
            overlaps[i][j] = get_overlap(N, ising_model, gamma, beta, sim, result)

    return overlaps

N = 10
G = nx.erdos_renyi_graph(N, 0.5, seed=42)  # has 21 edges
gammas = np.linspace(0, np.pi/2, 50)
betas = np.linspace(0, np.pi, 50)
overlaps = collect_parameter_data(G, gammas, betas)
_ = plot_heat_map(gammas, betas, overlaps, "True QAOA Overlap", "Gamma", "Beta")

In [ ]:
from QAOA_simulator import get_simulator, get_expectation, get_result
from plot_utils import plot_heat_map
import networkx as nx
import qokit.maxcut as mc
import numpy as np

def collect_parameter_data(G: nx.Graph, gammas: np.ndarray, betas: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    expectations = np.zeros([len(gammas), len(betas)])
    N = G.number_of_nodes()
    ising_model = mc.get_maxcut_terms(G)
    sim = get_simulator(N, ising_model)

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            result = get_result(N, ising_model, gamma, beta, sim)
            expectations[i][j] = get_expectation(N, ising_model, gamma, beta, sim, result)

    return expectations

N = 10
G = nx.erdos_renyi_graph(N, 0.5, seed=42)  # has 21 edges
gammas = np.linspace(0, np.pi/2, 50)
betas = np.linspace(0, np.pi, 50)
overlaps = collect_parameter_data(G, gammas, betas)
_ = plot_heat_map(gammas, betas, overlaps, "True Expectation", "Gamma", "Beta")

In [ ]:
from QAOA_simulator import get_simulator, get_expectation, get_overlap, get_result
from plot_utils import plot_heat_map
import networkx as nx
import qokit.maxcut as mc
import numpy as np

def collect_parameter_data(G: nx.Graph, gammas: np.ndarray, betas: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    expectations = np.zeros([len(gammas), len(betas)])
    overlaps = np.zeros([len(gammas), len(betas)])
    N = G.number_of_nodes()
    ising_model = mc.get_maxcut_terms(G)
    sim = get_simulator(N, ising_model)

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            result = get_result(N, ising_model, gamma, beta, sim)
            expectations[i][j] = get_expectation(N, ising_model, gamma, beta, sim, result)
            overlaps[i][j] = get_overlap(N, ising_model, gamma, beta, sim, result)


    return expectations, overlaps

N = 15
G = nx.erdos_renyi_graph(N, 0.5, seed=42)  # has 21 edges
gammas = np.linspace(0, np.pi/2, 50)
betas = np.linspace(0, np.pi, 50)
expectations, overlaps = collect_parameter_data(G, gammas, betas)
shifted_expectations = expectations - np.amin(expectations)
_ = plot_heat_map(gammas, betas, expectations, "True Expectation", "Gamma", "Beta")
_ = plot_heat_map(gammas, betas, overlaps, "True Overlap", "Gamma", "Beta")
_ = plot_heat_map(gammas, betas, (shifted_expectations/np.amax(shifted_expectations)) - (overlaps/np.amax(overlaps)), "True Expectation Minus Overlap", "Gamma", "Beta")

In [ ]:
from QAOA_simulator import get_simulator, get_overlap, get_expectation, get_result
from plot_utils import plot_heat_map
import networkx as nx
import qokit.maxcut as mc
import numpy as np
from QAOA_proxy import QAOA_proxy
from QAOA_paper_proxy import QAOA_paper_proxy


def collect_parameter_data(G: nx.Graph, gammas: np.ndarray, betas: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    expectations = np.zeros([len(gammas), len(betas)])
    overlaps = np.zeros([len(gammas), len(betas)])
    N = G.number_of_nodes()
    ising_model = mc.get_maxcut_terms(G)
    sim = get_simulator(N, ising_model)

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            result = get_result(N, ising_model, gamma, beta, sim)
            overlaps[i][j] = get_overlap(N, ising_model, gamma, beta, sim, result)
            expectations[i][j] = get_expectation(N, ising_model, gamma, beta, sim, result)

    return expectations, overlaps


def collect_parameter_data_proxy_less_dumb(num_constraints: int, num_qubits: int, gammas: np.ndarray, betas: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    expectations = np.zeros([len(gammas), len(betas)])
    altered_proxy = np.zeros([len(gammas), len(betas)])

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            _, expectations[i][j] = QAOA_proxy(1, gamma, beta, num_constraints, num_qubits)
            _, altered_proxy[i][j] = QAOA_proxy(1, gamma, beta, num_constraints, num_qubits, 2*(num_constraints+1)//3)

    return expectations, altered_proxy


def collect_parameter_data_proxy(num_constraints: int, num_qubits: int, gammas: np.ndarray, betas: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    expectations = np.zeros([len(gammas), len(betas)])

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            _, expectations[i][j] = QAOA_paper_proxy(1, gamma, beta, num_constraints, num_qubits, 2*(num_constraints+1)//3)

    return expectations


N = 10
G = nx.erdos_renyi_graph(N, 0.5, seed=42)  # has 10 edges
gammas = np.linspace(0, np.pi, 50)
betas = np.linspace(0, 2*np.pi, 50)
expectations, overlaps = collect_parameter_data(G, gammas, betas)
_ = plot_heat_map(gammas, betas, expectations, "True QAOA Expectations", "Gamma", "Beta")
_ = plot_heat_map(gammas, betas, overlaps, "True QAOA Overlap", "Gamma", "Beta")
expectation_proxies = collect_parameter_data_proxy(G.number_of_edges(), N, gammas, betas)
_ = plot_heat_map(gammas, betas, expectation_proxies, "Expectation Proxies from Paper", "Gamma", "Beta")
expectation_proxies_less_dumb, altered_less_dumb = collect_parameter_data_proxy_less_dumb(G.number_of_edges(), N, gammas, betas)
_ = plot_heat_map(gammas, betas, expectation_proxies_less_dumb, "Expectation Proxies from us", "Gamma", "Beta")
_ = plot_heat_map(gammas, betas, altered_less_dumb, "Altered Proxies from us", "Gamma", "Beta")

In [ ]:
import QAOA_proxy as qpld
import numpy as np


def test_plot(num_constraints: int, num_qubits: int) -> None:
    costs = np.linspace(0, num_constraints, num_constraints + 1)
    num_with_costs = np.zeros(len(costs))
    for i in range(len(costs)):
        num_with_costs[i] = qpld.number_with_cost_proxy(costs[i], num_constraints, num_qubits)

    fig = plt.figure()
    ax = fig.add_subplot()
    ax.scatter(costs, num_with_costs)


test_plot(20, 10)

In [ ]:
import QAOA_proxy
import numpy as np
from plot_utils import plot_3d_scatter
import QAOA_paper_proxy

%matplotlib widget

def test_plot_by_cost_paper(cost: int, num_constraints: int, num_qubits: int) -> None:
    cost_2s = np.linspace(0, num_constraints, num_constraints+1, dtype=int)
    distances = np.linspace(0, num_qubits, num_qubits+1, dtype=int)
    num_of_costs_at_distances = np.zeros([len(cost_2s), len(distances)])
    for i in range(len(cost_2s)):
        for j in range(len(distances)):
            num_of_costs_at_distances[i][j] = QAOA_paper_proxy.number_of_costs_at_distance_paper_proxy(cost, cost_2s[i], distances[j], num_constraints, num_qubits)
    
    plot_3d_scatter(cost_2s, distances, num_of_costs_at_distances, f"\"Approximate\" N({cost};d,c) from paper", "cost_2", "distance", "proxy")

def test_plot_by_cost(cost: int, num_constraints: int, num_qubits: int) -> None:
    cost_2s = np.linspace(0, num_constraints, num_constraints+1, dtype=int)
    distances = np.linspace(0, num_qubits, num_qubits+1, dtype=int)
    num_of_costs_at_distances = np.zeros([len(cost_2s), len(distances)])
    for i in range(len(cost_2s)):
        for j in range(len(distances)):
            num_of_costs_at_distances[i][j] = QAOA_proxy.number_of_costs_at_distance_proxy(cost, cost_2s[i], distances[j], num_constraints, num_qubits)
    
    plot_3d_scatter(cost_2s, distances, num_of_costs_at_distances, f"\"Approximate\" N({cost};d,c) from us", "cost_2", "distance", "proxy")

test_plot_by_cost_paper(8, 20, 10)
test_plot_by_cost(8, 20, 10)

In [ ]:
from QAOA_paper_proxy import number_of_costs_at_distance_paper_proxy
import numpy as np
from plot_utils import plot_3d_scatter

%matplotlib widget

def test_plot(num_constraints_max: int, num_qubits_max: int) -> None:
    num_constraints = np.linspace(0, num_constraints_max, num_constraints_max + 1, dtype=int)
    num_qubits = np.linspace(0, num_qubits_max, num_qubits_max + 1, dtype=int)
    peak_hs = np.zeros([len(num_constraints), len(num_qubits)])
    for c in range(len(num_constraints)):
        for q in range(len(num_qubits)):
            try:
                peak_hs[c][q] = number_of_costs_at_distance_paper_proxy(c//2, c//2, q//2, c, q)
            except:
                ()
    
    plot_3d_scatter(num_constraints[7:], num_qubits, peak_hs[7:], f"Peak values from Paper", "num_constraints", "num_qubits", "peak value")

test_plot(15, 12)

In [ ]:
import numpy as np
from plot_utils import plot_3d_scatter

%matplotlib widget

def test_plot(num_constraints_max: int, num_qubits_max: int) -> None:
    num_constraints = np.linspace(0, num_constraints_max, num_constraints_max + 1, dtype=int)
    num_qubits = np.linspace(0, num_qubits_max, num_qubits_max + 1, dtype=int)
    peak_hs = np.zeros([len(num_constraints), len(num_qubits)])
    for c in range(len(num_constraints)):
        for q in range(len(num_qubits)):
            try:
                peak_hs[c][q] = 2**(q-4)
            except:
                ()
    plot_3d_scatter(num_constraints, num_qubits, peak_hs, f"Peak values from us", "num_constraints", "num_qubits", "peak value_proxy")

test_plot(15, 12)

In [ ]:
import networkx as nx

N = 10
nx.erdos_renyi_graph(N, 0.5, seed=42).number_of_edges()